In [1]:
import json

file_path = '../data/annotated_github_topics_wikidata.json'

In [2]:
qids = []
with open(file_path, 'rt') as inf:
    for line in inf:
        obj = json.loads(line)
        qids.append(obj['Wikidata ID'])

In [3]:
from wikidata.client import Client

client = Client()

types = []
for q in qids:
    entity = client.get(q, load=True)
    #entity_types = entity.data['claims']['P31']
    t = []
    try:
        entity_types = entity.data['claims']['P31']
    except:
        types.append(t)
        continue

    for etype in entity_types:
        try:
            type_id = etype['mainsnak']['datavalue']['value']['id']
            type_title = client.get(type_id, load=True).label.texts['en']
            t.append((type_id, type_title))
            types.append(t)
        except:
            types.append(t)
            continue

In [4]:
list(zip(qids[:100], types[:100]))

[('Q102190569', [('Q13582682', 'classification')]),
 ('Q1041418', []),
 ('Q104537917', []),
 ('Q105273027', [('Q17155032', 'software category')]),
 ('Q1061077', []),
 ('Q1061635', [('Q268592', 'industry')]),
 ('Q1068715', []),
 ('Q1070689', []),
 ('Q107300377', []),
 ('Q1074173', [('Q4485156', 'software feature')]),
 ('Q107630377', []),
 ('Q107643461', [('Q11862829', 'academic discipline')]),
 ('Q107750691', []),
 ('Q1078276', []),
 ('Q10861030', [('Q105675274', 'regression')]),
 ('Q10927554', []),
 ('Q11023',
  [('Q627436', 'field of work'),
   ('Q3958441', 'economic sector'),
   ('Q2465832', 'branch of science'),
   ('Q11862829', 'academic discipline'),
   ('Q28797', 'applied science')]),
 ('Q1123036',
  [('Q627436', 'field of work'),
   ('Q3958441', 'economic sector'),
   ('Q2465832', 'branch of science'),
   ('Q11862829', 'academic discipline'),
   ('Q28797', 'applied science')]),
 ('Q1127410',
  [('Q627436', 'field of work'),
   ('Q3958441', 'economic sector'),
   ('Q2465832', 'br

In [5]:
test_set = set()
for i in types:
    for x, names in i:
        test_set.add(x)

In [6]:
from collections import defaultdict, Counter

file_path = '../data/reconciled-gittopic.json'

i = 0
new_terms = []
duplicates = defaultdict(list)
count = Counter()
with open(file_path, 'rt') as inf:
    for line in inf:
        obj = json.loads(line)
        candidates = obj['candidates']
        for candidate in candidates:
            candidate_types = set(candidate['types'].keys())
            intersection = test_set.intersection(candidate_types)
            if intersection:
                new_terms.append(obj['term'] + "\n")
                break
            #if candidate['id'] in qids and obj['frequency'] > 10:
            #    i += 1
            #    duplicates[candidate['id']].append(obj['term'])
            #    count[candidate['id']] += obj['frequency']

In [7]:
with open('taxonomy_like_topics.csv', 'wt') as outf:
    outf.writelines(new_terms)